# Security Knowledge Base - Sigma, MITRE ATT&CK, OWASP

Notebook này load và query các documents từ:
1. **Sigma Rules** - Detection rules từ YAML files
2. **MITRE ATT&CK Techniques** - Từ trang enterprise
3. **OWASP Cheatsheets** - Từ thư mục local

## 📋 Thứ tự chạy các cells:

### Bước 1: Setup
- **Cell 1**: Imports
- **Cell 2**: Helper Functions

### Bước 2: Load Documents
- **Cell 3-5**: Load và Parse Sigma Rules
- **Cell 6-7**: Load MITRE ATT&CK Techniques (từ trang enterprise)
- **Cell 8**: Load OWASP Cheatsheets (từ thư mục local)

### Bước 3: Text Splitting
- **Cell 9**: Text Splitting và Combine All Documents
- **Cell 10**: Xem Chunks và Metadata

### Bước 4: Embedding & ChromaDB
- **Cell 11**: Embedding Model Setup
- **Cell 12**: ChromaDB Setup (embed tất cả documents)

### Bước 5: Query & Test
- **Cell 13**: Balanced Query (cân bằng kết quả từ mỗi loại)
- **Cell 14**: Balanced Query Test

### Bước 6: Hybrid Retrieval & Reranking (Nâng cao)
- **Cell 15**: Setup Reranker và BM25
- **Cell 16**: Hybrid Retrieval Function (Semantic + Keyword)
- **Cell 17**: Reranking Function
- **Cell 18**: Complete Pipeline (Hybrid + Rerank)
- **Cell 19**: Test Hybrid Retrieval + Reranking

**Lưu ý:** Cần cài đặt:
- `pip install rank-bm25 flashrank` cho hybrid retrieval và reranking


In [ ]:
# Imports
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import WebBaseLoader, AsyncChromiumLoader
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_text_splitters import RecursiveCharacterTextSplitter
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except ImportError:
    from langchain_community.embeddings import HuggingFaceEmbeddings

# Reranker và Hybrid Retrieval
try:
    from rank_bm25 import BM25Okapi
except ImportError:
    print("⚠️ rank_bm25 chưa được cài đặt. Chạy: pip install rank-bm25")
    BM25Okapi = None

# Reranker: Sử dụng Cross-Encoder từ sentence-transformers (tương thích với embedding model)
try:
    from sentence_transformers import CrossEncoder
    CROSS_ENCODER_AVAILABLE = True
except ImportError:
    CROSS_ENCODER_AVAILABLE = False
    print("⚠️ sentence-transformers chưa có CrossEncoder. Sẽ dùng simple reranking.")
    CrossEncoder = None

# Fallback: FlashRank hoặc Cohere
try:
    from flashrank import Ranker, RerankRequest
    FLASHRANK_AVAILABLE = True
except ImportError:
    FLASHRANK_AVAILABLE = False

try:
    from langchain_cohere import CohereRerank
    COHERE_AVAILABLE = True
except ImportError:
    COHERE_AVAILABLE = False

import os
import yaml
import json
import requests
import glob
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import numpy as np
from typing import List, Tuple, Dict
from collections import Counter


In [73]:
# Helper Functions

def parse_chroma_metadata(doc: Document) -> dict:
    """
    Parse metadata từ ChromaDB, convert JSON strings về lại list/dict
    """
    metadata = doc.metadata.copy()
    json_fields = ['tags', 'references', 'detection', 'logsource', 'detection_keywords', 'falsepositives']
    
    for field in json_fields:
        if field in metadata and isinstance(metadata[field], str):
            try:
                metadata[field] = json.loads(metadata[field])
            except (json.JSONDecodeError, TypeError):
                pass  # Giữ nguyên nếu không parse được
    
    return metadata

def format_doc_for_llm(doc: Document, include_full_rule: bool = False) -> str:
    """
    Format document với metadata để LLM dễ đọc
    """
    metadata = parse_chroma_metadata(doc)
    
    output = []
    output.append(f"Title: {metadata.get('title', 'N/A')}")
    output.append(f"ID: {metadata.get('id', 'N/A')}")
    output.append(f"Status: {metadata.get('status', 'N/A')}")
    output.append(f"Level: {metadata.get('level', 'N/A')}")
    output.append(f"Description: {metadata.get('description', 'N/A')}")
    
    if metadata.get('author'):
        output.append(f"Author: {metadata.get('author')}")
    
    if metadata.get('tags'):
        tags = metadata['tags'] if isinstance(metadata['tags'], list) else []
        output.append(f"Tags: {', '.join(str(t) for t in tags)}")
    
    if metadata.get('logsource'):
        logsource = metadata['logsource'] if isinstance(metadata['logsource'], dict) else {}
        output.append(f"Log Source: {json.dumps(logsource, ensure_ascii=False)}")
    
    if metadata.get('detection'):
        detection = metadata['detection'] if isinstance(metadata['detection'], dict) else {}
        if 'keywords' in detection and detection['keywords']:
            keywords = detection['keywords']
            keywords_preview = keywords[:5] if len(keywords) > 5 else keywords
            output.append(f"Detection Keywords: {', '.join(str(k) for k in keywords_preview)}")
            if len(keywords) > 5:
                output.append(f"  (+ {len(keywords)-5} more keywords)")
        if 'condition' in detection:
            output.append(f"Detection Condition: {detection['condition']}")
    
    if metadata.get('references'):
        refs = metadata['references'] if isinstance(metadata['references'], list) else []
        if refs:
            output.append(f"References: {len(refs)} reference(s)")
            for ref in refs[:3]:
                output.append(f"  - {ref}")
            if len(refs) > 3:
                output.append(f"  ... và {len(refs)-3} reference(s) khác")
    
    output.append(f"\nContent:\n{doc.page_content}")
    
    if include_full_rule and metadata.get('full_rule'):
        output.append(f"\nFull Rule YAML:\n{metadata['full_rule']}")
    
    return "\n".join(output)


## 1. Load và Parse Sigma Rules


In [74]:
# Đường dẫn tương đối từ notebook
sigma_path = os.path.join(os.path.dirname(os.getcwd()), "test", "sigma", "rules", "web", "webserver_generic")
# Hoặc đường dẫn tuyệt đối
if not os.path.exists(sigma_path):
    sigma_path = r"D:\MCPLLM\test\sigma\rules\web\webserver_generic"

# Kiểm tra thư mục có tồn tại không
if not os.path.exists(sigma_path):
    print(f"Error: Thư mục không tồn tại: {sigma_path}")
    print(f"Đường dẫn tuyệt đối: {os.path.abspath(sigma_path)}")
    sigma_raw_docs = []
else:
    try:
        sigma_loader = DirectoryLoader(
            path=sigma_path,
            glob="**/*.yml",
            loader_cls=TextLoader,
            loader_kwargs={'encoding': 'utf-8'}
        )
        sigma_raw_docs = sigma_loader.load()
        print(f"✅ Đã load {len(sigma_raw_docs)} file(s)")
        # In thông tin một vài file đầu tiên
        for i, doc in enumerate(sigma_raw_docs[:3]):
            print(f"\nFile {i+1}: {doc.metadata.get('source', 'N/A')}")
            print(f"Độ dài nội dung: {len(doc.page_content)} ký tự")
    except Exception as e:
        print(f"❌ Lỗi khi load: {e}")
        sigma_raw_docs = []


✅ Đã load 13 file(s)

File 1: d:\MCPLLM\test\sigma\rules\web\webserver_generic\web_f5_tm_utility_bash_api_request.yml
Độ dài nội dung: 1091 ký tự

File 2: d:\MCPLLM\test\sigma\rules\web\webserver_generic\web_iis_tilt_shortname_scan.yml
Độ dài nội dung: 902 ký tự

File 3: d:\MCPLLM\test\sigma\rules\web\webserver_generic\web_java_payload_in_access_logs.yml
Độ dài nội dung: 1191 ký tự


In [75]:
# Xử lý YAML và tạo processed docs
sigma_docs_processed = []

if sigma_raw_docs:
    print("="*50)
    print("Xử lý và parse YAML...")
    print("="*50)
    
    for doc in sigma_raw_docs:
        try:
            parsed_yaml = yaml.safe_load(doc.page_content)
            
            if parsed_yaml:
                # Tạo summary content chi tiết hơn
                title = parsed_yaml.get('title', 'N/A')
                description = parsed_yaml.get('description', 'N/A')
                level = parsed_yaml.get('level', 'N/A')
                status = parsed_yaml.get('status', 'N/A')
                
                summary_content = f"Sigma Rule: {title}\nStatus: {status} | Level: {level}\nDescription: {description}"
                
                # Extract detection keywords nếu có
                detection = parsed_yaml.get('detection', {})
                keywords = detection.get('keywords', []) if isinstance(detection, dict) else []
                if keywords:
                    keywords_preview = keywords[:3] if len(keywords) > 3 else keywords
                    summary_content += f"\nKeywords: {', '.join(str(k) for k in keywords_preview)}"
                    if len(keywords) > 3:
                        summary_content += f" (+{len(keywords)-3} more)"
                
                new_doc = Document(
                    page_content=summary_content,
                    metadata={
                        "source": doc.metadata.get('source'),
                        "full_rule": doc.page_content,
                        # Thông tin cơ bản
                        "title": parsed_yaml.get('title'),
                        "id": parsed_yaml.get('id'),
                        "status": parsed_yaml.get('status'),
                        "level": parsed_yaml.get('level'),
                        "description": parsed_yaml.get('description'),
                        # Thông tin tác giả và ngày tháng
                        "author": parsed_yaml.get('author'),
                        "date": str(parsed_yaml.get('date', '')),
                        "modified": str(parsed_yaml.get('modified', '')),
                        # Tags và categories
                        "tags": parsed_yaml.get('tags', []),
                        # Logsource
                        "logsource": parsed_yaml.get('logsource', {}),
                        # Detection rules
                        "detection": parsed_yaml.get('detection', {}),
                        "detection_keywords": keywords,
                        "detection_keywords_count": len(keywords),
                        # References và false positives
                        "references": parsed_yaml.get('references', []),
                        "falsepositives": parsed_yaml.get('falsepositives', []),
                    }
                )
                sigma_docs_processed.append(new_doc)
            else:
                print(f"Warning: Không parse được YAML từ {doc.metadata.get('source')}")
        except Exception as e:
            print(f"Lỗi khi parse YAML từ {doc.metadata.get('source')}: {e}")
    
    print(f"\n✅ Đã xử lý {len(sigma_docs_processed)} document(s)")
else:
    print("\n⚠️ Không có documents nào để xử lý.")


Xử lý và parse YAML...

✅ Đã xử lý 13 document(s)


## 2. Hiển thị Metadata (Optional)


In [76]:
# Xem một số docs đã processed (optional)
if sigma_docs_processed:
    for i, doc in enumerate(sigma_docs_processed[:2]):
        print(f"\n{'='*60}")
        print(f"--- Doc {i+1} ---")
        print(f"{'='*60}")
        print(f"\n📄 CONTENT:\n{doc.page_content}")
        print(f"\n📋 METADATA:")
        metadata_clean = {k: v for k, v in doc.metadata.items() if k != 'full_rule'}
        print(json.dumps(metadata_clean, indent=2, ensure_ascii=False))



--- Doc 1 ---

📄 CONTENT:
Sigma Rule: F5 BIG-IP iControl Rest API Command Execution - Webserver
Status: test | Level: medium
Description: Detects POST requests to the F5 BIG-IP iControl Rest API "bash" endpoint, which allows the execution of commands on the BIG-IP

📋 METADATA:
{
  "source": "d:\\MCPLLM\\test\\sigma\\rules\\web\\webserver_generic\\web_f5_tm_utility_bash_api_request.yml",
  "title": "F5 BIG-IP iControl Rest API Command Execution - Webserver",
  "id": "85254a62-22be-4239-b79c-2ec17e566c37",
  "status": "test",
  "level": "medium",
  "description": "Detects POST requests to the F5 BIG-IP iControl Rest API \"bash\" endpoint, which allows the execution of commands on the BIG-IP",
  "author": "Nasreddine Bencherchali (Nextron Systems), Thurein Oo",
  "date": "2023-11-08",
  "modified": "",
  "tags": [
    "attack.execution",
    "attack.t1190",
    "attack.initial-access"
  ],
  "logsource": {
    "category": "webserver"
  },
  "detection": {
    "selection": {
      "cs-met

## 3. Embedding và ChromaDB Setup


In [92]:
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [77]:
# Khởi tạo embedding model (dùng HuggingFace local)
print("📥 Đang tải embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Embedding model đã sẵn sàng")


📥 Đang tải embedding model...
✅ Embedding model đã sẵn sàng


In [78]:
# Tạo đường dẫn lưu ChromaDB
persist_directory = r"D:\MCPLLM\test\chroma_db"
os.makedirs(persist_directory, exist_ok=True)

# Chuẩn bị documents với metadata tương thích ChromaDB
print(f"💾 Đang tạo/load ChromaDB tại: {persist_directory}")

# Kiểm tra all_docs đã được tạo chưa
if 'all_docs' not in locals() or not all_docs:
    print("⚠️  Chưa có all_docs. Hãy chạy cell 'Text Splitting và Combine All Documents' trước.")
    print("    Đang dùng sigma_docs_processed tạm thời...")
    docs_to_process = sigma_docs_processed if 'sigma_docs_processed' in locals() and sigma_docs_processed else []
else:
    docs_to_process = all_docs

if not docs_to_process:
    print("❌ Không có documents nào để embed!")
    vectorstore = None
else:
    print(f"📝 Đang chuẩn bị {len(docs_to_process)} documents để embed...")
    
    # Convert list/dict trong metadata thành string cho ChromaDB
    docs_for_chroma = []
    for doc in docs_to_process:
        new_metadata = {}
        for key, value in doc.metadata.items():
            if isinstance(value, (list, dict)):
                # Convert list/dict thành JSON string
                new_metadata[key] = json.dumps(value, ensure_ascii=False)
            elif value is None:
                continue  # Skip None values
            else:
                new_metadata[key] = value
        
        # Tạo document mới với metadata đã xử lý
        new_doc = Document(
            page_content=doc.page_content,
            metadata=new_metadata
        )
        docs_for_chroma.append(new_doc)
    
    # Filter complex metadata một lần nữa để chắc chắn
    docs_for_chroma = filter_complex_metadata(docs_for_chroma)
    
    print(f"📊 Thống kê documents:")
    if 'all_docs' in locals() and all_docs:
        sigma_count = sum(1 for d in docs_for_chroma if d.metadata.get('source_type') == 'sigma_rule')
        mitre_count = sum(1 for d in docs_for_chroma if d.metadata.get('source_type') == 'mitre_attack')
        owasp_count = sum(1 for d in docs_for_chroma if d.metadata.get('source_type') == 'owasp_cheatsheet')
        print(f"   - Sigma rules: {sigma_count}")
        print(f"   - MITRE ATT&CK: {mitre_count}")
        print(f"   - OWASP cheatsheets: {owasp_count}")
    
    # Tạo ChromaDB vector store
    print(f"\n🔄 Đang embed và lưu vào ChromaDB...")
    vectorstore = Chroma.from_documents(
        documents=docs_for_chroma,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name="security_knowledge_base"  # Tên collection mới cho tất cả
    )
    print(f"✅ Đã lưu {len(docs_for_chroma)} documents vào ChromaDB")


💾 Đang tạo/load ChromaDB tại: D:\MCPLLM\test\chroma_db
📝 Đang chuẩn bị 3358 documents để embed...
📊 Thống kê documents:
   - Sigma rules: 13
   - MITRE ATT&CK: 13
   - OWASP cheatsheets: 3332

🔄 Đang embed và lưu vào ChromaDB...
✅ Đã lưu 3358 documents vào ChromaDB


In [89]:
print(f"Vectorstore created with {vectorstore._collection.count()} documents")


Vectorstore created with 3358 documents


In [90]:
# Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

# Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


In [91]:
sample_embedding

array([ 5.86870778e-03, -1.05241379e-02, -7.53515400e-04, -4.48323525e-02,
        1.37020694e-02, -5.52320853e-02, -4.34750579e-02, -2.44987961e-02,
       -1.93930585e-02,  6.64397404e-02, -3.66570130e-02, -4.55308333e-03,
       -2.87122764e-02, -5.15020937e-02,  4.28327583e-02,  4.38962458e-03,
        6.31853193e-02, -7.86684081e-02, -5.37194610e-02, -6.05001934e-02,
        3.53835449e-02, -1.62381884e-02, -1.12417797e-02, -1.24211917e-02,
       -1.33748949e-01,  1.00116376e-02, -5.86728416e-02, -3.09609696e-02,
       -2.88542069e-04, -9.58073884e-02, -3.07648070e-02, -7.13889953e-03,
       -8.03151447e-03,  5.32922857e-02,  1.12462575e-02,  6.91950396e-02,
        1.06334150e-01, -9.58079621e-02, -3.97749059e-03,  1.85524039e-02,
        2.87265740e-02, -7.92047828e-02, -2.40437035e-02, -2.27804184e-02,
        5.24348617e-02, -4.26563658e-02, -5.10982834e-02, -2.56277006e-02,
       -3.58699970e-02, -8.40453082e-04, -1.38057023e-02, -4.35068384e-02,
       -2.04492863e-02, -

In [95]:
# Lấy toàn bộ vector, tài liệu và metadata từ collection
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

# Đưa embedding vào mảng numpy
vectors = np.array(result['embeddings'])

# Lưu lại văn bản
documents = result['documents']

# Trích loại tài liệu từ metadata (giả sử có 'doc_type')
doc_types = [metadata['source_type'] for metadata in result['metadatas']]

# Gán màu sắc tùy theo loại tài liệu
colors = [['blue', 'green', 'red'][['sigma_rule', 'mitre_attack', 'owasp_cheatsheet'].index(t)] for t in doc_types]

In [96]:
# Con người chúng ta dễ hình dung mọi thứ trong không gian 2D hơn!
# Giảm số chiều của vector xuống 2D bằng t-SNE
# (T-distributed Stochastic Neighbor Embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Tạo biểu đồ scatter 2D
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Loại: {t}<br>Văn bản: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Biểu đồ 2D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show(renderer="browser")

In [84]:
## 9. Query và Test ChromaDB

# Test queries với các loại documents khác nhau
if 'vectorstore' in locals() and vectorstore:
    print("="*100)
    print("🔍 QUERY TEST - Tìm kiếm trong tất cả documents (Sigma, MITRE, OWASP)")
    print("="*100)
    
    # Hàm balanced query để cân bằng kết quả từ mỗi loại
    def balanced_query(vectorstore, query, k_per_source=3):
        """Query cân bằng: lấy kết quả từ mỗi source type riêng biệt"""
        all_results = []
        source_types = ['sigma_rule', 'mitre_attack', 'owasp_cheatsheet']
        
        for source_type in source_types:
            try:
                results = vectorstore.similarity_search_with_score(
                    query=query,
                    k=k_per_source,
                    filter={"source_type": source_type}
                )
                all_results.extend(results)
            except Exception as e:
                print(f"  ⚠️ Lỗi khi query {source_type}: {e}")
        
        # Sắp xếp lại theo score (score càng thấp càng tốt)
        all_results.sort(key=lambda x: x[1])
        return all_results
    
    # Test Query 1: SQL Injection
    print("\n\n[Query 1] SQL Injection")
    print("-"*100)
    query1 = "SQL injection attack detection"
    results1 = balanced_query(vectorstore, query1, k_per_source=3)
    print(f"🔎 Query: '{query1}'")
    print(f"📊 Tìm thấy {len(results1)} kết quả\n")
    
    for i, (doc, score) in enumerate(results1, 1):
        source_type = doc.metadata.get('source_type', 'unknown')
        print(f"[{i}] Score: {score:.4f} | Source: {source_type}")
        
        if source_type == 'sigma_rule':
            print(f"    Title: {doc.metadata.get('title', 'N/A')}")
            print(f"    Level: {doc.metadata.get('level', 'N/A')}")
        elif source_type == 'mitre_attack':
            print(f"    Technique: {doc.metadata.get('technique_id', 'N/A')} - {doc.metadata.get('technique_name', 'N/A')}")
        elif source_type == 'owasp_cheatsheet':
            print(f"    Cheatsheet: {doc.metadata.get('doc_type', 'N/A')}")
        
        # Preview content
        content_preview = doc.page_content if len(doc.page_content) > 200 else doc.page_content
        print(f"    Preview: {content_preview}...")
        print()
    
    # Test Query 2: Authentication
    print("\n\n[Query 2] Authentication Security")
    print("-"*100)
    query2 = "authentication security best practices"
    results2 = vectorstore.similarity_search_with_score(query2, k=5)
    print(f"🔎 Query: '{query2}'")
    print(f"📊 Tìm thấy {len(results2)} kết quả\n")
    
    for i, (doc, score) in enumerate(results2, 1):
        source_type = doc.metadata.get('source_type', 'unknown')
        print(f"[{i}] Score: {score:.4f} | Source: {source_type}")
        
        if source_type == 'sigma_rule':
            print(f"    Title: {doc.metadata.get('title', 'N/A')}")
        elif source_type == 'mitre_attack':
            print(f"    Technique: {doc.metadata.get('technique_id', 'N/A')} - {doc.metadata.get('technique_name', 'N/A')}")
        elif source_type == 'owasp_cheatsheet':
            print(f"    Cheatsheet: {doc.metadata.get('doc_type', 'N/A')}")
        
        content_preview = doc.page_content[:200] if len(doc.page_content) > 200 else doc.page_content
        print(f"    Preview: {content_preview}...")
        print()
    
    # Test Query 3: Credential Dumping
    print("\n\n[Query 3] Credential Dumping")
    print("-"*100)
    query3 = "credential dumping technique"
    results3 = vectorstore.similarity_search_with_score(query3, k=5)
    print(f"🔎 Query: '{query3}'")
    print(f"📊 Tìm thấy {len(results3)} kết quả\n")
    
    for i, (doc, score) in enumerate(results3, 1):
        source_type = doc.metadata.get('source_type', 'unknown')
        print(f"[{i}] Score: {score:.4f} | Source: {source_type}")
        
        if source_type == 'sigma_rule':
            print(f"    Title: {doc.metadata.get('title', 'N/A')}")
        elif source_type == 'mitre_attack':
            print(f"    Technique: {doc.metadata.get('technique_id', 'N/A')} - {doc.metadata.get('technique_name', 'N/A')}")
        elif source_type == 'owasp_cheatsheet':
            print(f"    Cheatsheet: {doc.metadata.get('doc_type', 'N/A')}")
        
        content_preview = doc.page_content if len(doc.page_content) > 200 else doc.page_content
        print(f"    Preview: {content_preview}...")
        print()
    
    # Test Query với Metadata Filtering
    print("\n\n[Query 4] Filter by Source Type - MITRE ATT&CK only")
    print("-"*100)
    query4 = "data obfuscation"
    results4 = vectorstore.similarity_search(
        query=query4,
        k=3,
        filter={"source_type": "mitre_attack"}
    )
    print(f"🔎 Query: '{query4}' (chỉ MITRE ATT&CK)")
    print(f"📊 Tìm thấy {len(results4)} kết quả\n")
    
    for i, doc in enumerate(results4, 1):
        print(f"[{i}] Technique: {doc.metadata.get('technique_id', 'N/A')} - {doc.metadata.get('technique_name', 'N/A')}")
        content_preview = doc.page_content[:300] if len(doc.page_content) > 300 else doc.page_content
        print(f"    Content: {content_preview}...")
        print()
    
    print("="*100)
    print("✅ HOÀN TẤT QUERY TEST")
    print("="*100)
    
else:
    print("⚠️ Chưa có vectorstore. Hãy chạy cell ChromaDB Setup trước.")


🔍 QUERY TEST - Tìm kiếm trong tất cả documents (Sigma, MITRE, OWASP)


[Query 1] SQL Injection
----------------------------------------------------------------------------------------------------
🔎 Query: 'SQL injection attack detection'
📊 Tìm thấy 9 kết quả

[1] Score: 0.4950 | Source: owasp_cheatsheet
    Cheatsheet: Injection_Prevention_Cheat_Sheet.md
    Preview: An SQL injection attack consists of insertion or "injection" of either a partial or complete SQL query via the data input or transmitted from the client (browser) to the web application.

A successful SQL injection attack can read sensitive data from the database, modify database data (insert/update/delete), execute administration operations on the database (such as shutdown the DBMS), recover the content of a given file existing on the DBMS file system or write files into the file system, and, in some cases, issue commands to the operating system. SQL injection attacks are a type of injection attack, in which SQL commands

In [85]:
## 9.1. Hàm Balanced Query (Cân bằng kết quả từ mỗi loại)

def balanced_query(vectorstore, query, k_per_source=3):
    """
    Query cân bằng: lấy kết quả từ mỗi source type riêng biệt
    Tránh trường hợp OWASP chiếm hết kết quả do có quá nhiều chunks
    
    Args:
        vectorstore: ChromaDB vector store
        query: Query string
        k_per_source: Số kết quả lấy từ mỗi loại (default: 3)
    
    Returns:
        List of (doc, score) tuples, đã sắp xếp theo score
    """
    all_results = []
    source_types = ['sigma_rule', 'mitre_attack', 'owasp_cheatsheet']
    
    for source_type in source_types:
        try:
            results = vectorstore.similarity_search_with_score(
                query=query,
                k=k_per_source,
                filter={"source_type": source_type}
            )
            all_results.extend(results)
        except Exception as e:
            print(f"  ⚠️ Lỗi khi query {source_type}: {e}")
    
    # Sắp xếp lại theo score (score càng thấp càng tốt trong ChromaDB)
    all_results.sort(key=lambda x: x[1])
    
    return all_results

print("✅ Hàm balanced_query đã được định nghĩa")
print("\n💡 Sử dụng:")
print("   results = balanced_query(vectorstore, 'SQL injection', k_per_source=3)")
print("   # Sẽ lấy 3 kết quả từ Sigma, 3 từ MITRE, 3 từ OWASP")


✅ Hàm balanced_query đã được định nghĩa

💡 Sử dụng:
   results = balanced_query(vectorstore, 'SQL injection', k_per_source=3)
   # Sẽ lấy 3 kết quả từ Sigma, 3 từ MITRE, 3 từ OWASP


In [86]:
## 9.2. Query Test với Balanced Query

# Đảm bảo hàm balanced_query đã được định nghĩa (chạy cell trước)
if 'vectorstore' in locals() and vectorstore:
    print("="*100)
    print("🔍 QUERY TEST - Cân bằng kết quả từ Sigma, MITRE, OWASP")
    print("="*100)
    
    # Test Query 1: SQL Injection
    print("\n\n[Query 1] SQL Injection")
    print("-"*100)
    query1 = "SQL injection attack detection"
    results1 = balanced_query(vectorstore, query1, k_per_source=3)
    print(f"🔎 Query: '{query1}'")
    print(f"📊 Tìm thấy {len(results1)} kết quả (3 từ mỗi loại)\n")
    
    # Thống kê theo source type
    sigma_count = sum(1 for d, _ in results1 if d.metadata.get('source_type') == 'sigma_rule')
    mitre_count = sum(1 for d, _ in results1 if d.metadata.get('source_type') == 'mitre_attack')
    owasp_count = sum(1 for d, _ in results1 if d.metadata.get('source_type') == 'owasp_cheatsheet')
    print(f"📊 Phân bố: Sigma={sigma_count}, MITRE={mitre_count}, OWASP={owasp_count}\n")
    
    for i, (doc, score) in enumerate(results1[:9], 1):  # Hiển thị tối đa 9 (3 từ mỗi loại)
        source_type = doc.metadata.get('source_type', 'unknown')
        print(f"[{i}] Score: {score:.4f} | Source: {source_type}")
        
        if source_type == 'sigma_rule':
            print(f"    Title: {doc.metadata.get('title', 'N/A')}")
            print(f"    Level: {doc.metadata.get('level', 'N/A')}")
        elif source_type == 'mitre_attack':
            print(f"    Technique: {doc.metadata.get('technique_id', 'N/A')} - {doc.metadata.get('technique_name', 'N/A')}")
        elif source_type == 'owasp_cheatsheet':
            print(f"    Cheatsheet: {doc.metadata.get('doc_type', 'N/A')}")
        
        # Preview content
        content_preview = doc.page_content[:150] if len(doc.page_content) > 150 else doc.page_content
        print(f"    Preview: {content_preview}...")
        print()
    
    # Test Query 2: XSS
    print("\n\n[Query 2] XSS Attack")
    print("-"*100)
    query2 = "cross-site scripting XSS prevention"
    results2 = balanced_query(vectorstore, query2, k_per_source=3)
    print(f"🔎 Query: '{query2}'")
    print(f"📊 Tìm thấy {len(results2)} kết quả (3 từ mỗi loại)\n")
    
    # Thống kê
    sigma_count2 = sum(1 for d, _ in results2 if d.metadata.get('source_type') == 'sigma_rule')
    mitre_count2 = sum(1 for d, _ in results2 if d.metadata.get('source_type') == 'mitre_attack')
    owasp_count2 = sum(1 for d, _ in results2 if d.metadata.get('source_type') == 'owasp_cheatsheet')
    print(f"📊 Phân bố: Sigma={sigma_count2}, MITRE={mitre_count2}, OWASP={owasp_count2}\n")
    
    for i, (doc, score) in enumerate(results2[:9], 1):
        source_type = doc.metadata.get('source_type', 'unknown')
        print(f"[{i}] Score: {score:.4f} | Source: {source_type}")
        
        if source_type == 'sigma_rule':
            print(f"    Title: {doc.metadata.get('title', 'N/A')}")
        elif source_type == 'mitre_attack':
            print(f"    Technique: {doc.metadata.get('technique_id', 'N/A')} - {doc.metadata.get('technique_name', 'N/A')}")
        elif source_type == 'owasp_cheatsheet':
            print(f"    Cheatsheet: {doc.metadata.get('doc_type', 'N/A')}")
        print()
    
    print("="*100)
    print("✅ HOÀN TẤT QUERY TEST")
    print("="*100)
    print("\n💡 Lưu ý: Query cân bằng đảm bảo có kết quả từ cả 3 loại (Sigma, MITRE, OWASP)")
    print("   Tránh trường hợp OWASP chiếm hết do có quá nhiều chunks.")
    
else:
    print("⚠️ Chưa có vectorstore. Hãy chạy cell ChromaDB Setup trước.")
    print("⚠️ Hoặc chưa định nghĩa balanced_query. Hãy chạy cell 'Hàm Balanced Query' trước.")


🔍 QUERY TEST - Cân bằng kết quả từ Sigma, MITRE, OWASP


[Query 1] SQL Injection
----------------------------------------------------------------------------------------------------
🔎 Query: 'SQL injection attack detection'
📊 Tìm thấy 9 kết quả (3 từ mỗi loại)

📊 Phân bố: Sigma=3, MITRE=3, OWASP=3

[1] Score: 0.4950 | Source: owasp_cheatsheet
    Cheatsheet: Injection_Prevention_Cheat_Sheet.md
    Preview: An SQL injection attack consists of insertion or "injection" of either a partial or complete SQL query via the data input or transmitted from the clie...

[2] Score: 0.5301 | Source: owasp_cheatsheet
    Cheatsheet: Injection_Prevention_Cheat_Sheet.md
    Preview: SQL Injection attacks can be divided into the following three classes:

- **Inband:**Â data is extracted using the same channel that is used to inject...

[3] Score: 0.5837 | Source: owasp_cheatsheet
    Cheatsheet: SQL_Injection_Prevention_Cheat_Sheet.md
    Preview: ## What Is a SQL Injection Attack?

Attackers can use SQ

In [10]:
# Format document cho LLM
sample_query = "Java payload attack"
sample_results = vectorstore.similarity_search(sample_query, k=1)

if sample_results:
    doc = sample_results[0]
    print("📝 Document formatted for LLM:")
    print("="*60)
    formatted = format_doc_for_llm(doc, include_full_rule=False)
    print(formatted)


📝 Document formatted for LLM:
Title: Java Payload Strings
ID: 583aa0a2-30b1-4d62-8bf3-ab73689efe6c
Status: test
Level: high
Description: Detects possible Java payloads in web access logs
Author: frack113, Harjot Singh, "@cyb3rjy0t" (update)
Tags: cve.2022-26134, cve.2021-26084, attack.initial-access, attack.t1190
Log Source: {"category": "webserver"}
Detection Keywords: %24%7B%28%23a%3D%40, ${(#a=@, %24%7B%40java, ${@java, u0022java
  (+ 5 more keywords)
Detection Condition: keywords
References: 5 reference(s)
  - https://www.rapid7.com/blog/post/2022/06/02/active-exploitation-of-confluence-cve-2022-26134/
  - https://www.rapid7.com/blog/post/2021/09/02/active-exploitation-of-confluence-server-cve-2021-26084/
  - https://github.com/httpvoid/writeups/blob/62d3751945289d088ccfdf4d0ffbf61598a2cd7d/Confluence-RCE.md
  ... và 2 reference(s) khác

Content:
Sigma Rule: Java Payload Strings
Status: test | Level: high
Description: Detects possible Java payloads in web access logs
Keywords: %24%

In [11]:
# Query với metadata filtering
print("🔍 Tìm rules có level='high':")
high_level_docs = vectorstore.similarity_search(
    query="security detection",
    k=5,
    filter={"level": "high"}
)
print(f"Tìm thấy {len(high_level_docs)} rules với level=high")
for doc in high_level_docs[:3]:
    print(f"  - {doc.metadata.get('title', 'N/A')} (Level: {doc.metadata.get('level', 'N/A')})")


🔍 Tìm rules có level='high':
Tìm thấy 5 rules với level=high
  - Suspicious Windows Strings In URI (Level: high)
  - Suspicious Windows Strings In URI (Level: high)
  - Suspicious Windows Strings In URI (Level: high)


## 5. Custom Query

Thử query của bạn ở đây:


## 6. Load MITRE ATT&CK Techniques


In [12]:
# Load MITRE ATT&CK Techniques từ web
def get_mitre_technique_urls(base_url="https://attack.mitre.org/techniques/enterprise/"):
    """Lấy danh sách URL của các techniques từ MITRE ATT&CK"""
    try:
        response = requests.get(base_url, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        urls = set()
        # Tìm tất cả links đến techniques (có pattern /techniques/Txxxx)
        for link in soup.find_all('a', href=True):
            href = link['href']
            if '/techniques/T' in href:
                full_url = urljoin(base_url, href)
                # Chỉ lấy techniques, không lấy sub-techniques (có .xxx)
                if '/techniques/T' in full_url and '.001' not in full_url:
                    urls.add(full_url)
        
        return sorted(list(urls))
    except Exception as e:
        print(f"❌ Lỗi khi lấy danh sách URLs: {e}")
        return []

print("📥 Đang lấy danh sách MITRE ATT&CK technique URLs...")
mitre_urls = get_mitre_technique_urls()
print(f"✅ Tìm thấy {len(mitre_urls)} techniques")

# Chỉ load một số techniques đầu tiên để test (có thể bỏ limit sau)
mitre_urls_sample = mitre_urls[:10]  # Test với 10 techniques đầu tiên
print(f"📝 Sẽ load {len(mitre_urls_sample)} techniques (sample)")


📥 Đang lấy danh sách MITRE ATT&CK technique URLs...
✅ Tìm thấy 691 techniques
📝 Sẽ load 10 techniques (sample)


In [19]:
print(mitre_urls_sample)

['https://attack.mitre.org/techniques/T1001', 'https://attack.mitre.org/techniques/T1001/001', 'https://attack.mitre.org/techniques/T1001/002', 'https://attack.mitre.org/techniques/T1001/003', 'https://attack.mitre.org/techniques/T1003', 'https://attack.mitre.org/techniques/T1003/001', 'https://attack.mitre.org/techniques/T1003/002', 'https://attack.mitre.org/techniques/T1003/003', 'https://attack.mitre.org/techniques/T1003/004', 'https://attack.mitre.org/techniques/T1003/005']


In [48]:
import requests
from bs4 import BeautifulSoup
from langchain_core.documents import Document
import time
import re

def scrape_mitre_technique(url):
    """
    Scrape thông tin chi tiết từng Technique từ MITRE ATT&CK
    Format tối ưu cho text splitting:
    T1548
    Abuse Elevation Control Mechanism
    [Description text...]
    """
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract Technique ID từ URL
        url_parts = url.rstrip('/').split('/')
        if 'techniques' in url_parts:
            tech_idx = url_parts.index('techniques')
            if tech_idx + 1 < len(url_parts):
                tech_id_raw = url_parts[tech_idx + 1]
                if len(url_parts) > tech_idx + 2:
                    sub_id = url_parts[tech_idx + 2]
                    technique_id = f"{tech_id_raw}.{sub_id}"
                else:
                    technique_id = tech_id_raw
            else:
                technique_id = None
        else:
            technique_id = None
        
        # Extract Technique Name từ h1
        h1 = soup.find('h1')
        if h1:
            title_full = h1.get_text(strip=True)
            title_full = re.sub(r'\s*[-|]\s*MITRE ATT&CK.*$', '', title_full, flags=re.IGNORECASE)
            title_full = re.sub(r'Sub-technique\s+\w+\.\d+\s*[:]\s*', '', title_full, flags=re.IGNORECASE)
            title_full = re.sub(r'Technique\s+\w+\s*[:]\s*', '', title_full, flags=re.IGNORECASE)
            technique_name = title_full.strip()
        else:
            technique_name = "Unknown"
        
        # Extract Description
        description = ""
        desc_div = soup.find('div', class_='description-body')
        if desc_div:
            description = desc_div.get_text(separator=' ', strip=True)
            # Clean up: loại bỏ [1], [2] references nhưng giữ nội dung
            description = re.sub(r'\[\d+\]', '', description)
            description = ' '.join(description.split())  # Normalize whitespace
        else:
            desc_divs = soup.find_all('div', class_='card-data')
            for div in desc_divs:
                if div.get_text(strip=True):
                    description = div.get_text(separator=' ', strip=True)
                    description = re.sub(r'\[\d+\]', '', description)
                    description = ' '.join(description.split())
                    break
        
        if not description and h1:
            next_elem = h1.find_next_sibling()
            while next_elem:
                if next_elem.name in ['p', 'div']:
                    text = next_elem.get_text(strip=True)
                    if text and len(text) > 50:
                        description = ' '.join(text.split())
                        break
                next_elem = next_elem.find_next_sibling()
        
        # Format theo cấu trúc file mẫu: ID\nName\nDescription
        # Tối ưu cho text splitting - mỗi technique là một document riêng
        if technique_id and technique_name and description:
            # Format: T1548\nName\nDescription (giống file mẫu)
            content = f"{description}"
        else:
            content = f"{description}" if description else ""
        
        return {
            'technique_id': technique_id,
            'technique_name': technique_name,
            'description': description,
            'url': url,
            'content': content
        }
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Load MITRE ATT&CK techniques với format tối ưu
print("🔄 Đang load MITRE ATT&CK techniques (format tối ưu cho text splitting)...")
mitre_docs = []

for i, url in enumerate(mitre_urls_sample, 1):
    url_id = url.rstrip('/').split('/')[-1]
    print(f"[{i}/{len(mitre_urls_sample)}] {url_id}", end=" ")
    
    result = scrape_mitre_technique(url)
    
    if result and result['content']:
        doc = Document(
            page_content=result['content'],
            metadata={
                'source_type': 'mitre_attack',
                'source_url': result['url'],
                'technique_id': result['technique_id'],
                'technique_name': result['technique_name'],
                'description_length': len(result['description']) if result['description'] else 0
            }
        )
        mitre_docs.append(doc)
        desc_len = len(result['description']) if result['description'] else 0
        print(f"✅ {desc_len} chars")
    else:
        print("❌")
    
    time.sleep(0.5)

print(f"\n✅ Loaded: {len(mitre_docs)} MITRE ATT&CK documents")
if mitre_docs:
    print(f"\n📄 Sample document (format tối ưu):")
    sample = mitre_docs[0]
    print(f"  Technique ID: {sample.metadata.get('technique_id', 'N/A')}")
    print(f"  Technique Name: {sample.metadata.get('technique_name', 'N/A')}")
    print(f"\n  Content (format giống file mẫu):")
    print(f"  {'-'*60}")
    print(sample.page_content)
    print(f"  {'-'*60}")

🔄 Đang load MITRE ATT&CK techniques (format tối ưu cho text splitting)...
[1/10] T1001 ✅ 465 chars
[2/10] 001 ✅ 419 chars
[3/10] 002 ✅ 452 chars
[4/10] 003 ✅ 855 chars
[5/10] T1003 ✅ 481 chars
[6/10] 001 ✅ 2174 chars
[7/10] 002 ✅ 847 chars
[8/10] 003 ✅ 710 chars
[9/10] 004 ✅ 437 chars
[10/10] 005 ✅ 1226 chars

✅ Loaded: 10 MITRE ATT&CK documents

📄 Sample document (format tối ưu):
  Technique ID: T1001
  Technique Name: Data Obfuscation

  Content (format giống file mẫu):
  ------------------------------------------------------------
Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide commands from being seen. This encompasses many methods, such as adding junk data to protocol traffic, using steganography, or impersonating legitimate proto

In [33]:
# Xem nội dung document chi tiết
def view_document(doc, doc_index=None):
    """Hiển thị nội dung document một cách chi tiết"""
    print("=" * 80)
    if doc_index is not None:
        print(f"📄 DOCUMENT #{doc_index}")
    else:
        print(f"📄 DOCUMENT")
    print("=" * 80)
    
    print("\n📋 METADATA:")
    print("-" * 80)
    for key, value in doc.metadata.items():
        if isinstance(value, str) and len(value) > 500:
            print(f"  {key}: {value[:500]}...")
        else:
            print(f"  {key}: {value}")
    
    print("\n📝 CONTENT:")
    print("-" * 80)
    print(doc.page_content)
    print("=" * 80)
    print(f"\n📊 Thống kê:")
    print(f"  - Content length: {len(doc.page_content)} chars")
    print(f"  - Content lines: {len(doc.page_content.splitlines())} lines")
    print(f"  - Metadata fields: {len(doc.metadata)}")

# Xem document đầu tiên
if 'mitre_docs' in locals() and mitre_docs:
    print("🔍 Xem document đầu tiên:\n")
    view_document(mitre_docs[0], doc_index=0)
else:
    print("⚠️ Chưa có mitre_docs. Hãy chạy cell load MITRE ATT&CK techniques trước.")

🔍 Xem document đầu tiên:

📄 DOCUMENT #0

📋 METADATA:
--------------------------------------------------------------------------------
  source_type: mitre_attack
  source_url: https://attack.mitre.org/techniques/T1001
  technique_id: T1001
  technique_name: Data Obfuscation
  description_length: 465

📝 CONTENT:
--------------------------------------------------------------------------------
T1001
Data Obfuscation
Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide commands from being seen. This encompasses many methods, such as adding junk data to protocol traffic, using steganography, or impersonating legitimate protocols.

📊 Thống kê:
  - Content length: 488 chars
  - Content lines: 3 lines
  - Metadata fields: 5


In [34]:
# Xem nhiều documents cùng lúc
if 'mitre_docs' in locals() and mitre_docs:
    print(f"📚 Tổng số documents: {len(mitre_docs)}\n")
    
    # Xem tất cả documents (tóm tắt)
    print("📋 DANH SÁCH TẤT CẢ DOCUMENTS:")
    print("=" * 80)
    for i, doc in enumerate(mitre_docs):
        tech_id = doc.metadata.get('technique_id', 'N/A')
        tech_name = doc.metadata.get('technique_name', doc.metadata.get('title', 'N/A'))
        content_len = len(doc.page_content)
        print(f"  [{i}] {tech_id:15} | {tech_name[:50]:50} | {content_len:6} chars")
    
    print("\n" + "=" * 80)
    print("\n💡 Để xem chi tiết một document cụ thể:")
    print("   view_document(mitre_docs[0])  # Xem document đầu tiên")
    print("   view_document(mitre_docs[1])  # Xem document thứ hai")
    print("   print(mitre_docs[0].page_content)  # Chỉ xem content")
    print("   print(mitre_docs[0].metadata)  # Chỉ xem metadata")
else:
    print("⚠️ Chưa có mitre_docs. Hãy chạy cell load MITRE ATT&CK techniques trước.")


📚 Tổng số documents: 10

📋 DANH SÁCH TẤT CẢ DOCUMENTS:
  [0] T1001           | Data Obfuscation                                   |    488 chars
  [1] T1001.001       | Data Obfuscation:Junk Data                         |    456 chars
  [2] T1001.002       | Data Obfuscation:Steganography                     |    493 chars
  [3] T1001.003       | Data Obfuscation:Protocol or Service Impersonation |    916 chars
  [4] T1003           | OS Credential Dumping                              |    509 chars
  [5] T1003.001       | OS Credential Dumping:LSASS Memory                 |   2219 chars
  [6] T1003.002       | OS Credential Dumping:Security Account Manager     |    904 chars
  [7] T1003.003       | OS Credential Dumping:NTDS                         |    747 chars
  [8] T1003.004       | OS Credential Dumping:LSA Secrets                  |    481 chars
  [9] T1003.005       | OS Credential Dumping:Cached Domain Credentials    |   1284 chars


💡 Để xem chi tiết một document cụ thể:
   v

In [35]:
# Xem document với format đã tối ưu
if 'mitre_docs' in locals() and mitre_docs:
    print(f"📚 Tổng số documents: {len(mitre_docs)}\n")
    print("📋 FORMAT MẪU (tối ưu cho text splitting):")
    print("=" * 80)
    
    # Hiển thị một vài documents để xem format
    for i, doc in enumerate(mitre_docs[:3]):
        print(f"\n[Document {i+1}]")
        print("-" * 80)
        print(doc.page_content)
        print(f"\nMetadata:")
        print(f"  - technique_id: {doc.metadata.get('technique_id')}")
        print(f"  - technique_name: {doc.metadata.get('technique_name')}")
        print("-" * 80)
    
    print("\n💡 Format này tối ưu cho text splitting vì:")
    print("   - Mỗi technique là một document riêng")
    print("   - Format: ID\\nName\\nDescription (dễ parse)")
    print("   - Text splitter sẽ tách theo từng technique tự nhiên")
else:
    print("⚠️ Chưa có mitre_docs. Hãy chạy cell load MITRE ATT&CK techniques trước.")


📚 Tổng số documents: 10

📋 FORMAT MẪU (tối ưu cho text splitting):

[Document 1]
--------------------------------------------------------------------------------
T1001
Data Obfuscation
Adversaries may obfuscate command and control traffic to make it more difficult to detect. Command and control (C2) communications are hidden (but not necessarily encrypted) in an attempt to make the content more difficult to discover or decipher and to make the communication less conspicuous and hide commands from being seen. This encompasses many methods, such as adding junk data to protocol traffic, using steganography, or impersonating legitimate protocols.

Metadata:
  - technique_id: T1001
  - technique_name: Data Obfuscation
--------------------------------------------------------------------------------

[Document 2]
--------------------------------------------------------------------------------
T1001.001
Data Obfuscation:Junk Data
Adversaries may add junk data to protocols used for command and 

In [63]:
# In ra TẤT CẢ documents để xem
print("=" * 100)
print("📚 XEM TẤT CẢ DOCUMENTS")
print("=" * 100)

# 1. Sigma Rules
if 'sigma_docs_processed' in locals() and sigma_docs_processed:
    print(f"\n\n{'='*100}")
    print(f"📋 SIGMA RULES ({len(sigma_docs_processed)} documents)")
    print("=" * 100)
    for i, doc in enumerate(sigma_docs_processed):
        print(f"\n[Sigma Doc {i+1}/{len(sigma_docs_processed)}]")
        print("-" * 100)
        print(f"Title: {doc.metadata.get('title', 'N/A')}")
        print(f"Level: {doc.metadata.get('level', 'N/A')}")
        print(f"Status: {doc.metadata.get('status', 'N/A')}")
        print(f"\nContent:")
        print(doc.page_content)
        print("-" * 100)
else:
    print("\n⚠️ Chưa có sigma_docs_processed")

# 2. MITRE ATT&CK Techniques
if 'mitre_docs' in locals() and mitre_docs:
    print(f"\n\n{'='*100}")
    print(f"📋 MITRE ATT&CK TECHNIQUES ({len(mitre_docs)} documents)")
    print("=" * 100)
    for i, doc in enumerate(mitre_docs):
        print(f"\n[MITRE Doc {i+1}/{len(mitre_docs)}]")
        print("-" * 100)
        print(f"Technique ID: {doc.metadata.get('technique_id', 'N/A')}")
        print(f"Technique Name: {doc.metadata.get('technique_name', 'N/A')}")
        print(f"Description Length: {doc.metadata.get('description_length', 0)} chars")
        print(f"\nContent:")
        print(doc.page_content)
        print("-" * 100)
else:
    print("\n⚠️ Chưa có mitre_docs")

# 3. OWASP Cheatsheets
if 'owasp_docs' in locals() and owasp_docs:
    print(f"\n\n{'='*100}")
    print(f"📋 OWASP CHEATSHEETS ({len(owasp_docs)} documents)")
    print("=" * 100)
    for i, doc in enumerate(owasp_docs):
        print(f"\n[OWASP Doc {i+1}/{len(owasp_docs)}]")
        print("-" * 100)
        print(f"Doc Type: {doc.metadata.get('doc_type', 'N/A')}")
        print(f"Source: {doc.metadata.get('source', 'N/A')}")
        print(f"Content Length: {len(doc.page_content)} chars")
        print(f"\nContent:")
        print(doc.page_content[:1000] if len(doc.page_content) > 1000 else doc.page_content)
        if len(doc.page_content) > 1000:
            print(f"\n... (truncated, total {len(doc.page_content)} chars)")
        print("-" * 100)
else:
    print("\n⚠️ Chưa có owasp_docs")

# 4. All Documents (sau khi combine)
if 'all_docs' in locals() and all_docs:
    print(f"\n\n{'='*100}")
    print(f"📋 ALL DOCUMENTS (Sau khi combine) ({len(all_docs)} documents)")
    print("=" * 100)
    for i, doc in enumerate(all_docs):
        print(f"\n[All Doc {i+1}/{len(all_docs)}]")
        print("-" * 100)
        print(f"Source Type: {doc.metadata.get('source_type', 'N/A')}")
        
        if doc.metadata.get('source_type') == 'mitre_attack':
            print(f"Technique ID: {doc.metadata.get('technique_id', 'N/A')}")
            print(f"Technique Name: {doc.metadata.get('technique_name', 'N/A')}")
        elif doc.metadata.get('source_type') == 'owasp_cheatsheet':
            print(f"Doc Type: {doc.metadata.get('doc_type', 'N/A')}")
        else:
            print(f"Title: {doc.metadata.get('title', 'N/A')}")
            print(f"Level: {doc.metadata.get('level', 'N/A')}")
        
        print(f"Content Length: {len(doc.page_content)} chars")
        print(f"\nContent:")
        print(doc.page_content[:1000] if len(doc.page_content) > 1000 else doc.page_content)
        if len(doc.page_content) > 1000:
            print(f"\n... (truncated, total {len(doc.page_content)} chars)")
        print("-" * 100)
else:
    print("\n⚠️ Chưa có all_docs (chạy cell Text Splitting và Combine để tạo all_docs)")

print(f"\n\n{'='*100}")
print("✅ HOÀN TẤT XEM TẤT CẢ DOCUMENTS")
print("=" * 100)


📚 XEM TẤT CẢ DOCUMENTS


📋 SIGMA RULES (13 documents)

[Sigma Doc 1/13]
----------------------------------------------------------------------------------------------------
Title: F5 BIG-IP iControl Rest API Command Execution - Webserver
Level: medium
Status: test

Content:
Sigma Rule: F5 BIG-IP iControl Rest API Command Execution - Webserver
Status: test | Level: medium
Description: Detects POST requests to the F5 BIG-IP iControl Rest API "bash" endpoint, which allows the execution of commands on the BIG-IP
----------------------------------------------------------------------------------------------------

[Sigma Doc 2/13]
----------------------------------------------------------------------------------------------------
Title: Successful IIS Shortname Fuzzing Scan
Level: medium
Status: test

Content:
Sigma Rule: Successful IIS Shortname Fuzzing Scan
Status: test | Level: medium
Description: When IIS uses an old .Net Framework it's possible to enumerate folders with the symbol "~"
-

## 7. Load OWASP Cheatsheets


In [65]:
# Load OWASP Cheatsheets từ thư mục local
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

import os, glob
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# Đường dẫn tới thư mục cheatsheets
cheatsheets_path = r"D:\MCPLLM\test\cheatsheets"

# Kiểm tra thư mục có tồn tại không
if not os.path.exists(cheatsheets_path):
    print(f"⚠️  Thư mục không tồn tại: {cheatsheets_path}")
    print(f"   Tạo thư mục hoặc kiểm tra đường dẫn")
    owasp_docs = []
else:
    # Dùng raw string để tránh lỗi escape
    folders = glob.glob(os.path.join(cheatsheets_path, "*"))
    
    # Thiết lập text loader kwargs
    # Thử dùng autodetect nếu có chardet, nếu không thì dùng utf-8
    try:
        import chardet
        text_loader_kwargs = {'autodetect_encoding': True}
    except ImportError:
        print("  ⚠️  Module 'chardet' chưa được cài đặt. Dùng encoding='utf-8' thay thế.")
        print("  💡 Để cài: pip install chardet")
        text_loader_kwargs = {'encoding': 'utf-8'}
    
    owasp_docs = []
    print(f"📂 Đang load OWASP cheatsheets từ: {cheatsheets_path}")
    
    for folder in folders:
        doc_type = os.path.basename(folder)
        
        try:
            if os.path.isdir(folder):
                loader = DirectoryLoader(
                    folder,
                    glob="**/*.md",  # ✅ luôn dùng dấu /
                    loader_cls=TextLoader,
                    loader_kwargs=text_loader_kwargs
                )
                folder_docs = loader.load()
                
            elif os.path.isfile(folder) and folder.endswith('.md'):
                loader = TextLoader(folder, **text_loader_kwargs)
                folder_docs = loader.load()
            else:
                continue  # Bỏ qua nếu không phải file .md
            
            for doc in folder_docs:
                doc.metadata["doc_type"] = doc_type
                doc.metadata["source_type"] = "owasp_cheatsheet"
                # Thêm source path vào metadata
                if 'source' not in doc.metadata:
                    doc.metadata["source"] = folder
                owasp_docs.append(doc)
            
            if folder_docs:
                print(f"  ✅ Loaded {len(folder_docs)} docs từ: {doc_type}")
        
        except Exception as e:
            print(f"  ⚠️  Lỗi khi load {folder}: {e}")
            continue
    
    print(f"\n✅ Total OWASP cheatsheet documents loaded: {len(owasp_docs)}")
    
    # Hiển thị một vài documents mẫu
    if owasp_docs:
        print(f"\n📋 Một vài documents mẫu:")
        for i, doc in enumerate(owasp_docs[:3]):
            doc_type = doc.metadata.get('doc_type', 'N/A')
            source = doc.metadata.get('source', 'N/A')
            if isinstance(source, str):
                source_name = os.path.basename(source)
            else:
                source_name = str(source)
            print(f"  [{i+1}] {doc_type} | {source_name} | {len(doc.page_content)} chars")


📂 Đang load OWASP cheatsheets từ: D:\MCPLLM\test\cheatsheets
  ✅ Loaded 1 docs từ: Abuse_Case_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Access_Control_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: AJAX_Security_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Attack_Surface_Analysis_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Authentication_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Authorization_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Authorization_Testing_Automation_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Automotive_Security.md
  ✅ Loaded 1 docs từ: Bean_Validation_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Browser_Extension_Vulnerabilities_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: C-Based_Toolchain_Hardening_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Choosing_and_Using_Security_Questions_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: CI_CD_Security_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Clickjacking_Defense_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Content_Security_Policy_Cheat_Sheet.md
  ✅ Loaded 1 docs từ: Cookie_Theft_Mitigation_Cheat_Sheet.md
  ✅ Loaded 1 

In [69]:
# In ra TẤT CẢ OWASP documents
if 'owasp_docs' in locals() and owasp_docs:
    print("=" * 100)
    print(f"📋 OWASP CHEATSHEETS - TẤT CẢ DOCUMENTS ({len(owasp_docs)} documents)")
    print("=" * 100)
    
    for i, doc in enumerate(owasp_docs):
        print(f"\n\n{'='*100}")
        print(f"[OWASP Doc {i+1}/{len(owasp_docs)}]")
        print("=" * 100)
        
        # Metadata
        print("\n📋 METADATA:")
        print("-" * 100)
        for key, value in doc.metadata.items():
            if isinstance(value, str) and len(value) > 200:
                print(f"  {key}: {value}...")
            else:
                print(f"  {key}: {value}")
        
        # Content
        print(f"\n📝 CONTENT ({len(doc.page_content)} chars):")
        print("-" * 100)
        print(doc.page_content)
        print("-" * 100)
        
        print(f"\n📊 Thống kê:")
        print(f"  - Content length: {len(doc.page_content)} chars")
        print(f"  - Content lines: {len(doc.page_content.splitlines())} lines")
        print(f"  - Metadata fields: {len(doc.metadata)}")
    
    print(f"\n\n{'='*100}")
    print(f"✅ Đã in {len(owasp_docs)} OWASP documents")
    print("=" * 100)
else:
    print("⚠️ Chưa có owasp_docs. Hãy chạy cell Load OWASP Cheatsheets trước.")

📋 OWASP CHEATSHEETS - TẤT CẢ DOCUMENTS (107 documents)


[OWASP Doc 1/107]

📋 METADATA:
----------------------------------------------------------------------------------------------------
  source: D:\MCPLLM\test\cheatsheets\Abuse_Case_Cheat_Sheet.md
  doc_type: Abuse_Case_Cheat_Sheet.md
  source_type: owasp_cheatsheet

📝 CONTENT (28002 chars):
----------------------------------------------------------------------------------------------------
# Abuse Case Cheat Sheet (Historical)

## Archive Statement

Reviewers have identified that abuse cases are rarely used in practice. Additionally, the material is presented as a "getting started tutorial" which isn't appropriate for the cheat sheet series.

## Introduction

Often when the security level of an application is mentioned in requirements, the following _expressions_ are met:

- _The application must be secure_.
- _The application must defend against all attacks targeting this category of application_.
- _The application must defend a

## 8. Text Splitting và Combine All Documents


In [87]:
## 9. Xem Chunks và Metadata (Sau khi Split)

# In ra các chunks với metadata để xem thử
print("="*100)
print("📋 XEM CÁC CHUNKS SAU KHI SPLIT")
print("="*100)

if 'all_docs' in locals() and all_docs:
    print(f"\n📊 Tổng số chunks: {len(all_docs)}\n")
    
    # Thống kê theo source type
    sigma_chunks = [d for d in all_docs if d.metadata.get('source_type') == 'sigma_rule']
    mitre_chunks = [d for d in all_docs if d.metadata.get('source_type') == 'mitre_attack']
    owasp_chunks = [d for d in all_docs if d.metadata.get('source_type') == 'owasp_cheatsheet']
    
    print(f"📈 Thống kê chunks:")
    print(f"   - Sigma rules: {len(sigma_chunks)} chunks")
    print(f"   - MITRE ATT&CK: {len(mitre_chunks)} chunks")
    print(f"   - OWASP cheatsheets: {len(owasp_chunks)} chunks")
    print()
    
    # Xem một vài chunks từ mỗi loại
    print("="*100)
    print("📄 SAMPLE CHUNKS VỚI METADATA")
    print("="*100)
    
    # 1. Sigma chunks
    if sigma_chunks:
        print(f"\n\n[1] SIGMA RULE CHUNKS (Sample {min(2, len(sigma_chunks))} chunks)")
        print("-"*100)
        for i, chunk in enumerate(sigma_chunks[:2]):
            print(f"\n[Sigma Chunk {i+1}]")
            print(f"Content Length: {len(chunk.page_content)} chars")
            print(f"Metadata: {json.dumps(chunk.metadata, indent=2, ensure_ascii=False, default=str)}")
            print(f"\nContent:")
            print(chunk.page_content[:300] if len(chunk.page_content) > 300 else chunk.page_content)
            if len(chunk.page_content) > 300:
                print(f"... (truncated, total {len(chunk.page_content)} chars)")
            print("-"*100)
    
    # 2. MITRE chunks
    if mitre_chunks:
        print(f"\n\n[2] MITRE ATT&CK CHUNKS (Sample {min(2, len(mitre_chunks))} chunks)")
        print("-"*100)
        for i, chunk in enumerate(mitre_chunks[:2]):
            print(f"\n[MITRE Chunk {i+1}]")
            print(f"Technique ID: {chunk.metadata.get('technique_id', 'N/A')}")
            print(f"Technique Name: {chunk.metadata.get('technique_name', 'N/A')}")
            print(f"Content Length: {len(chunk.page_content)} chars")
            print(f"\nMetadata: {json.dumps(chunk.metadata, indent=2, ensure_ascii=False, default=str)}")
            print(f"\nContent:")
            print(chunk.page_content)
            print("-"*100)
    
    # 3. OWASP chunks
    if owasp_chunks:
        print(f"\n\n[3] OWASP CHEATSHEET CHUNKS (Sample {min(3, len(owasp_chunks))} chunks)")
        print("-"*100)
        for i, chunk in enumerate(owasp_chunks[:3]):
            print(f"\n[OWASP Chunk {i+1}]")
            print(f"Doc Type: {chunk.metadata.get('doc_type', 'N/A')}")
            print(f"Source: {chunk.metadata.get('source', 'N/A')}")
            print(f"Content Length: {len(chunk.page_content)} chars")
            print(f"\nMetadata: {json.dumps(chunk.metadata, indent=2, ensure_ascii=False, default=str)}")
            print(f"\nContent Preview (500 chars):")
            print(chunk.page_content[:500] if len(chunk.page_content) > 500 else chunk.page_content)
            if len(chunk.page_content) > 500:
                print(f"... (truncated, total {len(chunk.page_content)} chars)")
            print("-"*100)
    
    # Tóm tắt
    print("\n\n" + "="*100)
    print("📊 TỔNG KẾT CHUNKS")
    print("="*100)
    print(f"Tổng số chunks: {len(all_docs)}")
    print(f"  - Sigma: {len(sigma_chunks)} chunks")
    print(f"  - MITRE: {len(mitre_chunks)} chunks")
    print(f"  - OWASP: {len(owasp_chunks)} chunks")
    print(f"  - Unknown: {len(all_docs) - len(sigma_chunks) - len(mitre_chunks) - len(owasp_chunks)} chunks")
    
    # Thống kê kích thước
    avg_len = sum(len(d.page_content) for d in all_docs) / len(all_docs) if all_docs else 0
    min_len = min(len(d.page_content) for d in all_docs) if all_docs else 0
    max_len = max(len(d.page_content) for d in all_docs) if all_docs else 0
    print(f"\nKích thước chunks:")
    print(f"  - Trung bình: {avg_len:.0f} chars")
    print(f"  - Nhỏ nhất: {min_len} chars")
    print(f"  - Lớn nhất: {max_len} chars")
    
else:
    print("⚠️ Chưa có all_docs. Hãy chạy cell 'Text Splitting và Combine All Documents' trước.")


📋 XEM CÁC CHUNKS SAU KHI SPLIT

📊 Tổng số chunks: 3358

📈 Thống kê chunks:
   - Sigma rules: 13 chunks
   - MITRE ATT&CK: 13 chunks
   - OWASP cheatsheets: 3332 chunks

📄 SAMPLE CHUNKS VỚI METADATA


[1] SIGMA RULE CHUNKS (Sample 2 chunks)
----------------------------------------------------------------------------------------------------

[Sigma Chunk 1]
Content Length: 238 chars
Metadata: {
  "source": "d:\\MCPLLM\\test\\sigma\\rules\\web\\webserver_generic\\web_f5_tm_utility_bash_api_request.yml",
  "full_rule": "title: F5 BIG-IP iControl Rest API Command Execution - Webserver\nid: 85254a62-22be-4239-b79c-2ec17e566c37\nrelated:\n    - id: b59c98c6-95e8-4d65-93ee-f594dfb96b17\n      type: similar\nstatus: test\ndescription: Detects POST requests to the F5 BIG-IP iControl Rest API \"bash\" endpoint, which allows the execution of commands on the BIG-IP\nreferences:\n    - https://f5-sdk.readthedocs.io/en/latest/apidoc/f5.bigip.tm.util.html#module-f5.bigip.tm.util.bash\n    - https://co

In [70]:
# Text Splitting và Combine All Documents
# Tối ưu cho từng loại document: Sigma, MITRE, OWASP

# 1. Text splitter cho OWASP (Markdown files - có thể rất dài)
# OWASP cheatsheets thường là markdown, cần split để quản lý
owasp_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Phù hợp cho markdown content
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n## ", "\n\n### ", "\n\n", "\n", ". ", " ", ""]  # Ưu tiên split theo markdown headers
)

# 2. Text splitter cho MITRE (Format: T1548\nName\nDescription)
# Giữ nguyên technique nếu có thể, chỉ split nếu quá dài
mitre_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Đủ lớn cho hầu hết techniques
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". "]  # Ưu tiên giữ nguyên format ID\nName\nDescription
)

# Combine tất cả documents
all_docs = []

# =================================================================
# 1. SIGMA RULES
# =================================================================
# Sigma rules đã được processed, format ngắn gọn, không cần split
if 'sigma_docs_processed' in locals() and sigma_docs_processed:
    for doc in sigma_docs_processed:
        # Đảm bảo metadata có source_type
        if 'source_type' not in doc.metadata:
            doc.metadata['source_type'] = 'sigma_rule'
        all_docs.append(doc)
    print(f"✅ Added {len(sigma_docs_processed)} Sigma rule documents (không split)")

# =================================================================
# 2. MITRE ATT&CK TECHNIQUES
# =================================================================
# Format: T1548\nName\nDescription
# Mỗi technique là một document riêng, chỉ split nếu description quá dài
if 'mitre_docs' in locals() and mitre_docs:
    mitre_split = []
    for doc in mitre_docs:
        # Đảm bảo metadata có source_type
        if 'source_type' not in doc.metadata:
            doc.metadata['source_type'] = 'mitre_attack'
        
        # Nếu description quá dài (>2000 chars), split theo paragraph
        if len(doc.page_content) > 2000:
            chunks = mitre_splitter.split_documents([doc])
            # Giữ metadata cho mỗi chunk
            for chunk in chunks:
                chunk.metadata['source_type'] = 'mitre_attack'
            mitre_split.extend(chunks)
        else:
            # Giữ nguyên document - mỗi technique là một chunk
            mitre_split.append(doc)
    
    all_docs.extend(mitre_split)
    long_docs = sum(1 for doc in mitre_docs if len(doc.page_content) > 2000)
    print(f"✅ Added {len(mitre_split)} MITRE ATT&CK documents (from {len(mitre_docs)} techniques)")
    if long_docs > 0:
        print(f"   - {long_docs} techniques đã được split do description quá dài")

# =================================================================
# 3. OWASP CHEATSHEETS
# =================================================================
# OWASP cheatsheets là markdown files, có thể rất dài, cần split
if 'owasp_docs' in locals() and owasp_docs:
    owasp_split = []
    for doc in owasp_docs:
        # Đảm bảo metadata có source_type
        if 'source_type' not in doc.metadata:
            doc.metadata['source_type'] = 'owasp_cheatsheet'
        
        # Split OWASP documents (markdown có thể rất dài)
        chunks = owasp_splitter.split_documents([doc])
        # Giữ metadata cho mỗi chunk
        for chunk in chunks:
            chunk.metadata['source_type'] = 'owasp_cheatsheet'
            # Giữ nguyên doc_type và source từ document gốc
            if 'doc_type' in doc.metadata:
                chunk.metadata['doc_type'] = doc.metadata['doc_type']
            if 'source' in doc.metadata:
                chunk.metadata['source'] = doc.metadata['source']
        owasp_split.extend(chunks)
    
    all_docs.extend(owasp_split)
    print(f"✅ Added {len(owasp_split)} OWASP cheatsheet document chunks (from {len(owasp_docs)} docs)")

# =================================================================
# TỔNG KẾT
# =================================================================
print(f"\n📊 Tổng cộng: {len(all_docs)} documents để embed và lưu vào ChromaDB")
print(f"\n💡 Chiến lược splitting:")
print(f"   - Sigma rules: Giữ nguyên (không split)")
print(f"   - MITRE techniques: Giữ nguyên, chỉ split nếu >2000 chars")
print(f"   - OWASP cheatsheets: Split theo markdown headers và paragraphs")


✅ Added 13 Sigma rule documents (không split)
✅ Added 13 MITRE ATT&CK documents (from 10 techniques)
   - 1 techniques đã được split do description quá dài
✅ Added 3332 OWASP cheatsheet document chunks (from 107 docs)

📊 Tổng cộng: 3358 documents để embed và lưu vào ChromaDB

💡 Chiến lược splitting:
   - Sigma rules: Giữ nguyên (không split)
   - MITRE techniques: Giữ nguyên, chỉ split nếu >2000 chars
   - OWASP cheatsheets: Split theo markdown headers và paragraphs


In [71]:
print(owasp_split[0])

page_content='# Abuse Case Cheat Sheet (Historical)

## Archive Statement

Reviewers have identified that abuse cases are rarely used in practice. Additionally, the material is presented as a "getting started tutorial" which isn't appropriate for the cheat sheet series.' metadata={'source': 'D:\\MCPLLM\\test\\cheatsheets\\Abuse_Case_Cheat_Sheet.md', 'doc_type': 'Abuse_Case_Cheat_Sheet.md', 'source_type': 'owasp_cheatsheet'}


In [88]:
## 10. Custom Query - Thử query của bạn

# Thay đổi query ở đây để tìm kiếm
if 'vectorstore' in locals() and vectorstore:
    my_query = "XSS cross-site scripting prevention"
    k_results = 5
    
    print(f"🔎 Custom Query: '{my_query}'")
    print(f"📊 Số kết quả: {k_results}")
    print("="*100)
    
    results = vectorstore.similarity_search_with_score(my_query, k=k_results)
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n[{i}] Score: {score:.4f}")
        print("-"*100)
        
        source_type = doc.metadata.get('source_type', 'unknown')
        print(f"Source Type: {source_type}")
        
        if source_type == 'sigma_rule':
            print(f"Title: {doc.metadata.get('title', 'N/A')}")
            print(f"Level: {doc.metadata.get('level', 'N/A')}")
            print(f"Status: {doc.metadata.get('status', 'N/A')}")
        elif source_type == 'mitre_attack':
            print(f"Technique ID: {doc.metadata.get('technique_id', 'N/A')}")
            print(f"Technique Name: {doc.metadata.get('technique_name', 'N/A')}")
        elif source_type == 'owasp_cheatsheet':
            print(f"Doc Type: {doc.metadata.get('doc_type', 'N/A')}")
            print(f"Source: {doc.metadata.get('source', 'N/A')}")
        
        print(f"\nContent:")
        print(doc.page_content[:500] if len(doc.page_content) > 500 else doc.page_content)
        if len(doc.page_content) > 500:
            print(f"... (truncated, total {len(doc.page_content)} chars)")
        print("-"*100)
else:
    print("⚠️ Chưa có vectorstore. Hãy chạy cell ChromaDB Setup trước.")


🔎 Custom Query: 'XSS cross-site scripting prevention'
📊 Số kết quả: 5

[1] Score: 0.3043
----------------------------------------------------------------------------------------------------
Source Type: owasp_cheatsheet
Doc Type: Cross_Site_Scripting_Prevention_Cheat_Sheet.md
Source: D:\MCPLLM\test\cheatsheets\Cross_Site_Scripting_Prevention_Cheat_Sheet.md

Content:
# Cross Site Scripting Prevention Cheat Sheet

## Introduction

This cheat sheet helps developers prevent XSS vulnerabilities.

Cross-Site Scripting (XSS) is a misnomer. Originally this term was derived from early versions of the attack that were primarily focused on stealing data cross-site. Since then, the term has widened to include injection of basically any content. XSS attacks are serious and can lead to account impersonation, observing user behaviour, loading external content, stealing se
... (truncated, total 731 chars)
-------------------------------------------------------------------------------------------------

In [67]:
# Thay đổi query ở đây
my_query = "SQL injection"
k_results = 3

results = vectorstore.similarity_search_with_score(my_query, k=k_results)

print(f"🔎 Query: '{my_query}'")
print(f"Kết quả: {len(results)} documents")
print("="*60)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n[{i}] Score: {score:.4f}")
    formatted = format_doc_for_llm(doc)
    print(formatted)
    print("-"*60)


🔎 Query: 'SQL injection'
Kết quả: 3 documents

[1] Score: 0.9444
Title: SQL Injection Strings In URI
ID: 5513deaf-f49a-46c2-a6c8-3f111b5cb453
Status: test
Level: high
Description: Detects potential SQL injection attempts via GET requests in access logs.
Author: Saw Win Naung, Nasreddine Bencherchali (Nextron Systems), Thurein Oo (Yoma Bank)
Tags: attack.initial-access, attack.t1190
Log Source: {"category": "webserver"}
Detection Keywords: @@version, %271%27%3D%271, =select , =select(, =select%20
  (+ 25 more keywords)
Detection Condition: selection and keywords and not 1 of filter_main_*
References: 5 reference(s)
  - https://www.acunetix.com/blog/articles/exploiting-sql-injection-example/
  - https://www.acunetix.com/blog/articles/using-logs-to-investigate-a-web-application-attack/
  - https://brightsec.com/blog/sql-injection-payloads/
  ... và 2 reference(s) khác

Content:
Sigma Rule: SQL Injection Strings In URI
Status: test | Level: high
Description: Detects potential SQL injection